### Naive Bayes Classifier - twitter data

#### Bag of Words Approach

In [ ]:
import twitter
import json

#CONSUMER_KEY = ''
#CONSUMER_SECRET = ''

#OAUTH_TOKEN = ''
#OAUTH_TOKEN_SECRET = ''


auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [ ]:
q1 = 'donald trump' 
count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
search_results = twitter_api.search.tweets(q=q1, count=count)
q1_texts = search_results['statuses']

# use a loop
num_iterations = 10

for i in range(num_iterations):
    params = {a:b for a,b in [x.split('=') for x in search_results['search_metadata']['next_results'][1:].split('&')]}
    max_id = int(params['max_id'])    
    search_results = twitter_api.search.tweets(q=q1, count=count, max_id=max_id)
    q1_texts += search_results['statuses']

In [ ]:
q1_texts = [status['text'] for status in q1_texts]
words_trump = [w.lower() for t in q1_texts for w in t.split()]

In [ ]:
q1_texts[:10]

In [ ]:
q2 = 'kardashian' 
count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
search_results = twitter_api.search.tweets(q=q2, count=count)
q2_tweets = search_results['statuses']

# use a loop
num_iterations = 10

for i in range(num_iterations):
    params = {a:b for a,b in [x.split('=') for x in search_results['search_metadata']['next_results'][1:].split('&')]}
    max_id = int(params['max_id'])    
    search_results = twitter_api.search.tweets(q=q2, count=count, max_id=max_id)
    q2_tweets += search_results['statuses']

In [ ]:
q2_texts = [status['text'] for status in q2_tweets]
words_kardashian = [w.lower() for t in q2_texts for w in t.split()]

In [ ]:
q2_texts[:10]

### prepare our featuresets

In [ ]:
import nltk
from nltk.corpus import stopwords
import re

stub = re.compile('[^A-Za-z]')
common_words = ['rt']

def bag_of_non_stopwords(text):
    # create array of words
    words = [stub.sub('', w).lower() for w in text.split()]

    # remove English stop words and common words
    finalwords = set(words) - set(stopwords.words('english')) - set(common_words)
    
    # format feature set data object
    featureset = dict([(word, True) for word in finalwords if not word.startswith('http') and len(word)>2])
    
    return featureset

bag_of_non_stopwords(q1_texts[30])

In [ ]:
from collections import defaultdict

def extract_features(text):
    return bag_of_non_stopwords(text)

label_features = defaultdict(list)

for tweet_text in q2_texts:
    features = extract_features(tweet_text)
    label_features['kardashian'].append(features)

In [ ]:
label_features['kardashian'][1]

In [ ]:
for tweet_text in q1_texts:
    features = extract_features(tweet_text)
    label_features['trump'].append(features)

In [ ]:
label_features['trump'][3]

In [ ]:
label_features.keys()

In [ ]:
# split the data into training / test sets -> 75/25% split
def split_label_feats(lfeats, split=0.75):
    train_feats = []
    test_feats = []

    for label, feats in lfeats.iteritems():
        cutoff = int(len(feats) * split)
        train_feats.extend([(feat, label) for feat in feats[:cutoff]])
        test_feats.extend([(feat, label) for feat in feats[cutoff:]])

    return train_feats, test_feats

In [ ]:
# Split data into training/testing - (split = 0.75) by default
train_feats, test_feats = split_label_feats(label_features)

print len(train_feats)
print len(test_feats)

In [ ]:
# Naive Bayes Classifier
from nltk.classify import NaiveBayesClassifier

nb = NaiveBayesClassifier.train(train_feats)

In [ ]:
ex1_features = extract_features("protect us from terrorists")
nb.classify(ex1_features)

In [ ]:
ex2_features = extract_features("pregnant with another baby !@#!@")
nb.classify(ex2_features)

In [ ]:
# Calculate accuracy
from nltk.classify.util import accuracy

accuracy(nb, test_feats)

In [ ]:
# getting classification probability of each label
probs = nb.prob_classify(test_feats[6][0])

print probs.samples()
print 'chosen class:',probs.max()
print 'P(trump)=',probs.prob('trump')
print 'P(kardashian)=',probs.prob('kardashian')

In [ ]:
test_feats[6]

In [ ]:
# most informative feature - feature value will always be True

nb.most_informative_features(10)

In [ ]:
# more information on the chosen label

nb.show_most_informative_features(10)